## 조건부 엣지를 활용한 비즈니스 이메일 챗봇

목표
- 비즈니스 제안 이메일에 대해 수락/거절 응답을 작성하는 챗봇을 구현합니다.
- 조건부 엣지를 활용하여 수락/거절에 따른 다른 응답 경로를 구성합니다.
- Human in the Loop 시스템을 통해 작성된 응답을 검토하고 수정할 수 있습니다.

시나리오
- 파트너사(GlobalTech)로부터 비즈니스 제안 이메일이 도착했습니다.
- 제안 내용을 분석하고 수락/거절을 결정합니다.
- 결정에 따라 적절한 응답을 자동으로 작성합니다.
- 담당자가 최종 검토 후 발송합니다.

샘플 이메일
```
보내는 사람: james@globaltech.com
제목: AI 솔루션 파트너십 제안

안녕하세요,

저희 GlobalTech는 귀사와 AI 솔루션 분야에서 전략적 파트너십을 제안드립니다.

제안 내용:
1. 공동 제품 개발 (LangGraph 기반 엔터프라이즈 솔루션)
2. 수익 배분: 50:50
3. 초기 투자금: 각 사 $500,000
4. 프로젝트 기간: 2년
5. 독점 계약 조건 포함

이 제안에 대한 귀사의 의견을 듣고 싶습니다.

감사합니다.
James Park
CEO, GlobalTech
```

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)

# 프로젝트 이름
logging.langsmith("LangGraph-Business-Email-Bot")

In [ ]:
# 준비 코드
from typing import Annotated, Literal
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt
from langchain_teddynote.messages import stream_graph
from langchain_teddynote.graphs import visualize_graph

### 1. State 정의 및 모델 설정

- 비즈니스 이메일 챗봇을 위한 State를 정의하세요.
- State에는 messages, proposal_analysis, decision, email_draft, review_status가 포함되어야 합니다.
- ChatOpenAI 모델(gpt-4.1, temperature=0.3)을 생성하세요.

In [ ]:
# 실습 코드
# TODO: State를 정의하세요.

class BusinessEmailState(TypedDict):
    messages: # 코드 입력: add_messages 사용
    proposal_analysis: # 코드 입력: str 타입 (제안 내용 분석)
    decision: # 코드 입력: Literal["pending", "accept", "reject"] 타입
    email_draft: # 코드 입력: str 타입
    review_status: # 코드 입력: Literal["draft", "approved", "needs_revision"] 타입
    human_feedback: # 코드 입력: str 타입

# TODO: LLM 모델을 정의하세요.
llm = # 코드 입력

In [ ]:
# 정답 코드
# TODO: State를 정의하세요.

class BusinessEmailState(TypedDict):
    messages: Annotated[list, add_messages]
    proposal_analysis: str  # 제안 내용 분석
    decision: Literal["pending", "accept", "reject"]  # 수락/거절 결정
    email_draft: str  # 이메일 초안
    review_status: Literal["draft", "approved", "needs_revision"]  # 검토 상태
    human_feedback: str  # 수정 요청 피드백

# TODO: LLM 모델을 정의하세요.
llm = ChatOpenAI(model="gpt-4.1", temperature=0.3)

### 2. 이메일 분석 노드 구현

- 받은 비즈니스 제안 이메일을 분석하는 노드를 구현하세요.
- 제안의 주요 내용을 요약하고 proposal_analysis에 저장하세요.
- decision을 "pending"으로 설정하세요.

In [ ]:
# 실습 코드
# TODO: 이메일 분석 노드를 구현하세요.

def analyze_proposal(state: BusinessEmailState):
    """비즈니스 제안 이메일을 분석합니다."""
    
    system_prompt = # 코드 입력: 제안 분석 지시
    
    messages = # 코드 입력: SystemMessage와 state의 messages 결합
    
    analysis = # 코드 입력: LLM 호출
    
    print("\n📊 제안 분석 완료:")
    print(analysis.content)
    
    return {
        # 코드 입력: proposal_analysis와 decision 반환
    }

In [ ]:
# 정답 코드
# TODO: 이메일 분석 노드를 구현하세요.

def analyze_proposal(state: BusinessEmailState):
    """비즈니스 제안 이메일을 분석합니다."""
    
    system_prompt = """당신은 비즈니스 제안을 분석하는 전문가입니다.
    다음 제안 이메일의 핵심 내용을 분석하여 요약하세요:
    1. 제안사 정보
    2. 주요 제안 내용
    3. 재무 조건
    4. 기간 및 조건
    5. 잠재적 리스크와 기회"""
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    
    analysis = llm.invoke(messages)
    
    print("\n📊 제안 분석 완료:")
    print(analysis.content)
    
    return {
        "proposal_analysis": analysis.content,
        "decision": "pending"
    }

### 3. 응답 결정 노드 구현

- 분석된 제안에 대해 수락/거절을 결정하는 노드를 구현하세요.
- interrupt를 사용하여 사용자의 결정을 받으세요.
- decision을 "accept" 또는 "reject"로 업데이트하세요.

In [ ]:
# 실습 코드
# TODO: 응답 결정 노드를 구현하세요.

def decide_response(state: BusinessEmailState):
    """제안에 대한 수락/거절을 결정합니다."""
    
    print("\n🤔 제안에 대한 결정이 필요합니다.")
    print("옵션: accept(수락) / reject(거절)")
    
    # interrupt를 호출하여 사용자 결정 대기
    user_decision = # 코드 입력: interrupt 호출
    
    decision = # 코드 입력: user_decision에서 action 가져오기
    
    print(f"\n✅ 결정: {decision}")
    
    return # 코드 입력: decision 반환

In [ ]:
# 정답 코드
# TODO: 응답 결정 노드를 구현하세요.

def decide_response(state: BusinessEmailState):
    """제안에 대한 수락/거절을 결정합니다."""
    
    print("\n🤔 제안에 대한 결정이 필요합니다.")
    print("옵션: accept(수락) / reject(거절)")
    
    # interrupt를 호출하여 사용자 결정 대기
    user_decision = interrupt({"analysis": state["proposal_analysis"]})
    
    decision = user_decision.get("action", "reject")
    
    print(f"\n✅ 결정: {decision}")
    
    return {"decision": decision}

### 4. 조건부 응답 작성 노드

- 수락/거절에 따라 다른 응답을 작성하는 두 개의 노드를 구현하세요.
- draft_accept_email: 수락 응답 작성
- draft_reject_email: 거절 응답 작성
- 각각 다른 톤과 내용으로 작성하도록 구현하세요.

In [ ]:
# 실습 코드
# TODO: 수락 응답 작성 노드를 구현하세요.

def draft_accept_email(state: BusinessEmailState):
    """제안을 수락하는 이메일을 작성합니다."""
    
    system_prompt = # 코드 입력: 수락 이메일 작성 지시
    
    # human_feedback이 있으면 반영
    if state.get("human_feedback"):
        system_prompt += # 코드 입력: 피드백 반영 지시
    
    messages = # 코드 입력: 메시지 구성
    
    response = # 코드 입력: LLM 호출
    
    return {
        # 코드 입력: email_draft와 review_status 반환
    }

# TODO: 거절 응답 작성 노드를 구현하세요.

def draft_reject_email(state: BusinessEmailState):
    """제안을 거절하는 이메일을 작성합니다."""
    
    system_prompt = # 코드 입력: 거절 이메일 작성 지시
    
    # human_feedback이 있으면 반영
    if state.get("human_feedback"):
        system_prompt += # 코드 입력: 피드백 반영 지시
    
    messages = # 코드 입력: 메시지 구성
    
    response = # 코드 입력: LLM 호출
    
    return {
        # 코드 입력: email_draft와 review_status 반환
    }

In [ ]:
# 정답 코드
# TODO: 수락 응답 작성 노드를 구현하세요.

def draft_accept_email(state: BusinessEmailState):
    """제안을 수락하는 이메일을 작성합니다."""
    
    system_prompt = """당신은 비즈니스 이메일 작성 전문가입니다.
    제안을 수락하는 긍정적이고 열정적인 응답을 작성하세요.
    다음 사항을 포함하세요:
    1. 제안에 대한 감사 인사
    2. 파트너십에 대한 기대와 열정
    3. 다음 단계 제안 (미팅, 계약서 검토 등)
    4. 연락처 정보
    
    원본 제안 분석: {}
    """.format(state.get("proposal_analysis", ""))
    
    # human_feedback이 있으면 반영
    if state.get("human_feedback"):
        system_prompt += f"\n\n[중요] 다음 피드백을 반영하세요: {state['human_feedback']}"
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    
    response = llm.invoke(messages)
    
    return {
        "email_draft": response.content,
        "review_status": "draft"
    }

# TODO: 거절 응답 작성 노드를 구현하세요.

def draft_reject_email(state: BusinessEmailState):
    """제안을 거절하는 이메일을 작성합니다."""
    
    system_prompt = """당신은 비즈니스 이메일 작성 전문가입니다.
    제안을 정중하게 거절하는 응답을 작성하세요.
    다음 사항을 포함하세요:
    1. 제안에 대한 감사 인사
    2. 거절 이유 (구체적이지 않아도 됨)
    3. 향후 기회에 대한 열린 자세
    4. 좋은 관계 유지 희망
    
    원본 제안 분석: {}
    """.format(state.get("proposal_analysis", ""))
    
    # human_feedback이 있으면 반영
    if state.get("human_feedback"):
        system_prompt += f"\n\n[중요] 다음 피드백을 반영하세요: {state['human_feedback']}"
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    
    response = llm.invoke(messages)
    
    return {
        "email_draft": response.content,
        "review_status": "draft"
    }

### 5. Human Review 노드 및 조건부 라우팅

- 작성된 이메일을 검토하는 Human Review 노드를 구현하세요.
- decision에 따라 수락/거절 이메일 작성 노드로 라우팅하는 함수를 구현하세요.
- review_status에 따라 다음 단계를 결정하는 라우팅 함수를 구현하세요.

In [ ]:
# 실습 코드
# TODO: Human Review 노드를 구현하세요.

def human_review(state: BusinessEmailState):
    """작성된 이메일을 검토합니다."""
    
    print("\n📧 이메일 초안 검토:")
    print("=" * 50)
    print(state["email_draft"])
    print("=" * 50)
    print("옵션: approve(승인) / revise(수정요청)")
    
    review = # 코드 입력: interrupt 호출
    
    if review.get("action") == "approve":
        return # 코드 입력: review_status를 "approved"로 설정
    else:
        return {
            # 코드 입력: human_feedback과 review_status 설정
        }

# TODO: decision 기반 라우팅 함수를 구현하세요.

def route_by_decision(state: BusinessEmailState) -> str:
    """결정에 따라 다음 노드를 선택합니다."""
    # 코드 입력: decision에 따라 "draft_accept" 또는 "draft_reject" 반환
    pass

# TODO: review_status 기반 라우팅 함수를 구현하세요.

def route_by_review(state: BusinessEmailState) -> str:
    """검토 상태에 따라 다음 노드를 선택합니다."""
    # 코드 입력: review_status에 따라 적절한 노드 반환
    pass

# TODO: 이메일 발송 노드를 구현하세요.

def send_email(state: BusinessEmailState):
    """최종 이메일을 발송합니다."""
    # 코드 입력: 발송 완료 메시지 출력 및 반환
    pass

In [ ]:
# 정답 코드
# TODO: Human Review 노드를 구현하세요.

def human_review(state: BusinessEmailState):
    """작성된 이메일을 검토합니다."""
    
    print("\n📧 이메일 초안 검토:")
    print("=" * 50)
    print(state["email_draft"])
    print("=" * 50)
    print("옵션: approve(승인) / revise(수정요청)")
    
    review = interrupt({"draft": state["email_draft"]})
    
    if review.get("action") == "approve":
        return {"review_status": "approved"}
    else:
        return {
            "human_feedback": review.get("feedback", "다시 작성해주세요."),
            "review_status": "needs_revision"
        }

# TODO: decision 기반 라우팅 함수를 구현하세요.

def route_by_decision(state: BusinessEmailState) -> str:
    """결정에 따라 다음 노드를 선택합니다."""
    if state["decision"] == "accept":
        return "draft_accept"
    else:
        return "draft_reject"

# TODO: review_status 기반 라우팅 함수를 구현하세요.

def route_by_review(state: BusinessEmailState) -> str:
    """검토 상태에 따라 다음 노드를 선택합니다."""
    status = state.get("review_status", "draft")
    
    if status == "approved":
        return "send_email"
    elif status == "needs_revision":
        # 수정이 필요한 경우 decision에 따라 다시 작성
        if state["decision"] == "accept":
            return "draft_accept"
        else:
            return "draft_reject"
    else:  # draft
        return "human_review"

# TODO: 이메일 발송 노드를 구현하세요.

def send_email(state: BusinessEmailState):
    """최종 이메일을 발송합니다."""
    decision_type = "수락" if state["decision"] == "accept" else "거절"
    print(f"\n✉️ {decision_type} 이메일이 발송되었습니다!")
    print("\n최종 이메일:")
    print("=" * 50)
    print(state["email_draft"])
    print("=" * 50)
    
    return {"messages": [AIMessage(content=f"{decision_type} 이메일이 성공적으로 발송되었습니다.")]}

### 6. 그래프 구성 및 실행

- 모든 노드를 연결하여 완전한 그래프를 구성하세요.
- 조건부 엣지를 적절히 설정하여 수락/거절 경로를 구현하세요.
- 샘플 이메일로 그래프를 실행하고 테스트하세요.

In [ ]:
# 실습 코드
# TODO: 완전한 그래프를 구성하세요.

# 그래프 빌더 생성
builder = # 코드 입력: StateGraph 생성

# 노드 추가
builder.# 코드 입력: analyze_proposal 노드 추가
builder.# 코드 입력: decide_response 노드 추가
builder.# 코드 입력: draft_accept_email 노드 추가 (이름: "draft_accept")
builder.# 코드 입력: draft_reject_email 노드 추가 (이름: "draft_reject")
builder.# 코드 입력: human_review 노드 추가
builder.# 코드 입력: send_email 노드 추가

# 엣지 추가
builder.# 코드 입력: START → "analyze"
builder.# 코드 입력: "analyze" → "decide"
builder.# 코드 입력: "decide" → route_by_decision (조건부)
builder.# 코드 입력: "draft_accept" → route_by_review (조건부)
builder.# 코드 입력: "draft_reject" → route_by_review (조건부)
builder.# 코드 입력: "human_review" → route_by_review (조건부)
builder.# 코드 입력: "send_email" → END

# 체크포인터 설정 및 컴파일
memory = # 코드 입력: InMemorySaver 생성
app = # 코드 입력: checkpointer와 함께 컴파일

# 그래프 시각화
# 코드 입력: visualize_graph 호출

In [ ]:
# 정답 코드
# TODO: 완전한 그래프를 구성하세요.

# 그래프 빌더 생성
builder = StateGraph(BusinessEmailState)

# 노드 추가
builder.add_node("analyze", analyze_proposal)
builder.add_node("decide", decide_response)
builder.add_node("draft_accept", draft_accept_email)
builder.add_node("draft_reject", draft_reject_email)
builder.add_node("human_review", human_review)
builder.add_node("send_email", send_email)

# 엣지 추가
builder.add_edge(START, "analyze")
builder.add_edge("analyze", "decide")
builder.add_conditional_edges("decide", route_by_decision)
builder.add_conditional_edges("draft_accept", route_by_review)
builder.add_conditional_edges("draft_reject", route_by_review)
builder.add_conditional_edges("human_review", route_by_review)
builder.add_edge("send_email", END)

# 체크포인터 설정 및 컴파일
memory = InMemorySaver()
app = builder.compile(checkpointer=memory)

# 그래프 시각화
visualize_graph(app)

In [ ]:
# 실습 코드
# TODO: 샘플 이메일로 그래프를 실행하세요.
from langchain_core.runnables import RunnableConfig

# 샘플 비즈니스 제안 이메일
business_proposal = """보내는 사람: james@globaltech.com
제목: AI 솔루션 파트너십 제안

안녕하세요,

저희 GlobalTech는 귀사와 AI 솔루션 분야에서 전략적 파트너십을 제안드립니다.

제안 내용:
1. 공동 제품 개발 (LangGraph 기반 엔터프라이즈 솔루션)
2. 수익 배분: 50:50
3. 초기 투자금: 각 사 $500,000
4. 프로젝트 기간: 2년
5. 독점 계약 조건 포함

이 제안에 대한 귀사의 의견을 듣고 싶습니다.

감사합니다.
James Park
CEO, GlobalTech"""

# 설정
config = # 코드 입력: RunnableConfig 생성 (thread_id: "proposal_001")

# 그래프 실행
result = # 코드 입력: app.invoke 실행

In [ ]:
# 정답 코드
# TODO: 샘플 이메일로 그래프를 실행하세요.
from langchain_core.runnables import RunnableConfig

# 샘플 비즈니스 제안 이메일
business_proposal = """보내는 사람: james@globaltech.com
제목: AI 솔루션 파트너십 제안

안녕하세요,

저희 GlobalTech는 귀사와 AI 솔루션 분야에서 전략적 파트너십을 제안드립니다.

제안 내용:
1. 공동 제품 개발 (LangGraph 기반 엔터프라이즈 솔루션)
2. 수익 배분: 50:50
3. 초기 투자금: 각 사 $500,000
4. 프로젝트 기간: 2년
5. 독점 계약 조건 포함

이 제안에 대한 귀사의 의견을 듣고 싶습니다.

감사합니다.
James Park
CEO, GlobalTech"""

# 설정
config = RunnableConfig(configurable={"thread_id": "proposal_001"})

# 그래프 실행 (분석 단계)
result = app.invoke(
    {"messages": [HumanMessage(content=business_proposal)]},
    config=config
)

이제 interrupt 지점에서 Command를 사용하여 상호작용하세요:
1. 첫 번째 interrupt: 수락(accept) 또는 거절(reject) 결정
2. 두 번째 interrupt: 이메일 초안 승인(approve) 또는 수정 요청(revise)

In [ ]:
# 수락/거절 결정 (예시: 수락)
app.invoke(
    Command(resume={"action": "accept"}),
    config=config
)

In [ ]:
# 이메일 검토 (예시: 수정 요청)
app.invoke(
    Command(resume={
        "action": "revise",
        "feedback": "emoji 를 포함하여 친근감 있게 작성하세요."
    }),
    config=config
)

In [ ]:
# 수정된 이메일 승인
app.invoke(
    Command(resume={"action": "approve"}),
    config=config
)